In [2]:
import numpy as np
import pandas as pd
import os
import cv2
import torch
import torchvision

In [3]:
data_path = '/data/vision/polina/scratch/ruizhi/chestxray/data/png_16bit_256/'
save_path = '/data/vision/polina/projects/chestxray/ading/resnet_chestxray/data/48h_window/3_class/'
path = save_path + 'training_bnp_pairs.csv'
data = pd.read_csv(path)

In [4]:
data['patient_id'] = data.apply(lambda row: \
                               row['mimic_id_1'][1:9], axis=1)

In [5]:
patients = set()
for item in data['patient_id']:
    patients.add(item)
patient_df = pd.DataFrame(list(patients))
patient_df

,0
0,12142332
1,12013634
2,16425412
3,18379244
4,13339319
...,...
3222,18832095
3223,12885435
3224,16086976
3225,13429749


In [6]:
training_patients = patient_df.sample(frac=.95, replace=False)
validation_patients = training_patients.sample(frac=.05, replace=False)

training_patients = list(training_patients[0])
validation_patients = list(validation_patients[0])
print(len(training_patients), len(validation_patients), len(training_patients)-len(validation_patients))

for item in validation_patients:
    training_patients.remove(item)
print(len(training_patients), len(validation_patients))

3066 153 2913
2913 153


In [7]:
training_sample = data[data['patient_id'].isin(training_patients)]
training_sample

,mimic_id_1,mimic_id_2,delta_bnp,patient_id
0,p10000980_s59988438_925b9496-a956d7b2-05185e52...,p10000980_s51967283_943486a3-b3fa9ff7-50f5a769...,0,10000980
1,p10000980_s59988438_925b9496-a956d7b2-05185e52...,p10000980_s54980801_a75a1fbe-802065ad-717eb7c1...,0,10000980
2,p10000980_s59988438_925b9496-a956d7b2-05185e52...,p10000980_s57861150_5aa15ba6-55f5e96e-39cea686...,1,10000980
3,p10000980_s59988438_925b9496-a956d7b2-05185e52...,p10000980_s54935705_6ad819bb-bae74eb9-7b663e90...,2,10000980
4,p10000980_s59988438_925b9496-a956d7b2-05185e52...,p10000980_s50985099_6ad03ed1-97ee17ee-9cf8b320...,0,10000980
...,...,...,...,...
32043,p19998330_s59735820_05405a03-c11efc6e-8a4366b6...,p19998330_s58626532_6c6f0868-10028db0-0270ff74...,1,19998330
32044,p19998330_s59735820_05405a03-c11efc6e-8a4366b6...,p19998330_s50007325_b5f405fb-36876897-aee19743...,1,19998330
32045,p19998330_s52741948_c8b41e34-9295f63e-b2edb096...,p19998330_s58626532_6c6f0868-10028db0-0270ff74...,1,19998330
32046,p19998330_s52741948_c8b41e34-9295f63e-b2edb096...,p19998330_s50007325_b5f405fb-36876897-aee19743...,1,19998330


In [8]:
validation_sample = data[data['patient_id'].isin(validation_patients)]
validation_sample

,mimic_id_1,mimic_id_2,delta_bnp,patient_id
217,p10068741_s58252032_946771f8-9a60212c-410b6dcf...,p10068741_s52806019_c3a1fecb-cd098a70-60f60556...,1,10068741
228,p10078805_s52741797_fd90fdea-f6078759-4b840de9...,p10078805_s53458039_0eb018ae-b5a5cee9-1028cd0c...,2,10078805
229,p10078805_s52741797_fd90fdea-f6078759-4b840de9...,p10078805_s53458039_4588b6bc-a5e551ec-69f7fe68...,2,10078805
230,p10078805_s53458039_0eb018ae-b5a5cee9-1028cd0c...,p10078805_s53458039_4588b6bc-a5e551ec-69f7fe68...,1,10078805
231,p10088669_s57194095_f9ed00e1-1ff24595-61a4734e...,p10088669_s54662291_78deb463-d71515ff-678b3db6...,0,10088669
...,...,...,...,...
31510,p19826828_s56412303_6ad9a3f1-465dddb7-580da05e...,p19826828_s52119813_c4155fcf-87e3fe2d-55ae452c...,0,19826828
31519,p19840128_s53199022_3f28d7aa-f12dc831-739166d4...,p19840128_s51730766_9200140f-dc4234d9-fbbf00a7...,0,19840128
31705,p19864120_s57651391_ce452b7f-e96eaca9-19c90dba...,p19864120_s58395832_2b7a9c1e-23ccf51c-e06ec801...,1,19864120
31706,p19864120_s57651391_ce452b7f-e96eaca9-19c90dba...,p19864120_s53332977_661dcb6a-4e324725-732a0c82...,1,19864120


In [27]:
training_sample.to_csv(save_path + 'bnp_train.csv')
validation_sample.to_csv(save_path + 'bnp_validation.csv')


In [9]:
without_training = pd.concat([data, training_sample, validation_sample]).drop_duplicates(keep=False)
without_training

,mimic_id_1,mimic_id_2,delta_bnp,patient_id
49,p10013569_s53407104_2bf1576a-cb1775c8-7235d1b9...,p10013569_s57151026_63af38ce-8915b633-19f3a00b...,1,10013569
50,p10013569_s53407104_2bf1576a-cb1775c8-7235d1b9...,p10013569_s51292378_58d82a41-8b2938a0-054c9109...,1,10013569
51,p10013569_s53407104_2bf1576a-cb1775c8-7235d1b9...,p10013569_s54308488_172edbb7-5c4c96d8-4c5b4263...,1,10013569
52,p10013569_s53407104_2bf1576a-cb1775c8-7235d1b9...,p10013569_s56120093_99f17f78-e629010c-b8cc1364...,1,10013569
53,p10013569_s57151026_63af38ce-8915b633-19f3a00b...,p10013569_s51292378_58d82a41-8b2938a0-054c9109...,1,10013569
...,...,...,...,...
31516,p19839145_s52462743_d9a7a462-32a6f335-5ba91769...,p19839145_s51902131_a5504fed-522312c9-77c8dfeb...,1,19839145
31517,p19839145_s52462743_d9a7a462-32a6f335-5ba91769...,p19839145_s53000770_899d2c71-5b16892c-886ff786...,1,19839145
31518,p19839145_s51902131_a5504fed-522312c9-77c8dfeb...,p19839145_s53000770_899d2c71-5b16892c-886ff786...,1,19839145
31617,p19856485_s52172353_1ee2d35c-f1d77803-ced6fd7a...,p19856485_s53098626_492f0cb8-e6f1c107-96159f09...,0,19856485


In [29]:
without_training.to_csv(save_path + 'bnp_test.csv')

In [30]:
zeros = 0
ones = 0
twos = 0
threes = 0
for item in training_sample.iterrows():
    if item[-1]['delta_bnp'] == 0:
        zeros += 1
    elif item[-1]['delta_bnp'] == 1:
        ones += 1
    elif item[-1]['delta_bnp'] == 2:
        twos += 1
    else:
        threes += 1
print(zeros, ones, twos, threes)

7359 6914 7095 7254


## Analyzing Data

In [11]:
import pickle
import os

In [12]:
split_dir = '/data/vision/polina/projects/chestxray/ading/resnet_chestxray/data/48h_window/2_class'

train_split = pd.read_csv(split_dir + '/bnp_train.csv')
val_split = pd.read_csv(split_dir + '/bnp_validation.csv')
test_split = pd.read_csv(split_dir + '/bnp_test.csv')

In [13]:
len(val_split)

1288

In [14]:
splits = {'train':train_split, 'val':val_split, 'test':test_split}
split_ids = {}
for item in splits:
    ids = set()
    for row in splits[item].iterrows():
        patient_id = row[-1]['patient_id']
        ids.add(str(patient_id))
    split_ids[item] = ids
    

In [24]:
data_path = '/data/vision/polina/scratch/ruizhi/chestxray/data/png_16bit_256/'
save_path = '/data/vision/polina/projects/chestxray/ading/resnet_chestxray/data/48h_window/2_class/'
path = save_path + 'training_bnp_pairs.csv'
data = pd.read_csv(path)
data['patient_id'] = data.apply(lambda row: \
                               row['mimic_id_1'][1:9], axis=1)

In [25]:
train_data = data[data['patient_id'].isin(split_ids['train'])]
train_data

,mimic_id_1,mimic_id_2,delta_bnp,patient_id
0,p10000980_s59988438_925b9496-a956d7b2-05185e52...,p10000980_s51967283_943486a3-b3fa9ff7-50f5a769...,0,10000980
1,p10000980_s59988438_925b9496-a956d7b2-05185e52...,p10000980_s54980801_a75a1fbe-802065ad-717eb7c1...,0,10000980
2,p10000980_s59988438_925b9496-a956d7b2-05185e52...,p10000980_s57861150_5aa15ba6-55f5e96e-39cea686...,1,10000980
3,p10000980_s59988438_925b9496-a956d7b2-05185e52...,p10000980_s54935705_6ad819bb-bae74eb9-7b663e90...,1,10000980
4,p10000980_s59988438_925b9496-a956d7b2-05185e52...,p10000980_s50985099_6ad03ed1-97ee17ee-9cf8b320...,0,10000980
...,...,...,...,...
32022,p19997367_s53649096_c588dd1c-497b005f-61e4d5db...,p19997367_s55294872_2521d582-fe099130-01bf964d...,0,19997367
32023,p19997367_s53649096_c588dd1c-497b005f-61e4d5db...,p19997367_s50725718_58631769-31c4eeb4-fdbe4dce...,0,19997367
32024,p19997367_s58349138_65f2e144-53921f47-4ada7cec...,p19997367_s55294872_2521d582-fe099130-01bf964d...,1,19997367
32025,p19997367_s58349138_65f2e144-53921f47-4ada7cec...,p19997367_s50725718_58631769-31c4eeb4-fdbe4dce...,1,19997367


In [26]:
val_data = data[data['patient_id'].isin(split_ids['val'])]
val_data

,mimic_id_1,mimic_id_2,delta_bnp,patient_id
218,p10071869_s56653718_f385965c-93e349d2-22fde9ab...,p10071869_s53370316_d267070f-387a291c-c54081b3...,0,10071869
219,p10071869_s56653718_f385965c-93e349d2-22fde9ab...,p10071869_s54271988_3e00f15c-244f0c5f-5fee7b49...,0,10071869
220,p10071869_s56653718_f385965c-93e349d2-22fde9ab...,p10071869_s54202380_5e77b742-44f046b7-a33d39c8...,0,10071869
221,p10071869_s56653718_f385965c-93e349d2-22fde9ab...,p10071869_s54202380_d061fcc1-6996ddb4-baa01a3e...,0,10071869
222,p10071869_s53370316_d267070f-387a291c-c54081b3...,p10071869_s54271988_3e00f15c-244f0c5f-5fee7b49...,0,10071869
...,...,...,...,...
31431,p19809456_s52439316_a1f9b602-b91e65da-5ae6a395...,p19809456_s50022513_bf8ddd62-6cac2139-aed8a177...,1,19809456
31432,p19809456_s50022513_5feecef2-e806dc59-9c5b6fd4...,p19809456_s50022513_bf8ddd62-6cac2139-aed8a177...,0,19809456
31756,p19881566_s55497567_5532eccb-2f4ca7d3-3e05d8c0...,p19881566_s57964871_6c18a2bb-ae5009e0-54ed1415...,0,19881566
31776,p19896361_s54271299_fd18451d-48682067-602cf782...,p19896361_s58697305_0403b311-47b207e3-617ce1da...,1,19896361


In [27]:
test_data = data[data['patient_id'].isin(split_ids['test'])]
test_data

,mimic_id_1,mimic_id_2,delta_bnp,patient_id
59,p10013643_s55641691_691c93f1-584388ff-deffb3ba...,p10013643_s53841005_6e58e826-fc08e4ee-a9ff0245...,1,10013643
392,p10193065_s55346818_485d207a-4135a2cb-68687be0...,p10193065_s54435203_406ea27a-78b8b36d-023f550e...,0,10193065
393,p10193065_s55346818_485d207a-4135a2cb-68687be0...,p10193065_s58568609_de66d1a0-e150d1b3-146b4ac2...,1,10193065
394,p10193065_s55346818_485d207a-4135a2cb-68687be0...,p10193065_s57829729_a7573afa-f1706712-c3ee244b...,1,10193065
395,p10193065_s55346818_485d207a-4135a2cb-68687be0...,p10193065_s57829729_fdff70a3-1f820ce2-8890f98c...,1,10193065
...,...,...,...,...
32043,p19998330_s59735820_05405a03-c11efc6e-8a4366b6...,p19998330_s58626532_6c6f0868-10028db0-0270ff74...,1,19998330
32044,p19998330_s59735820_05405a03-c11efc6e-8a4366b6...,p19998330_s50007325_b5f405fb-36876897-aee19743...,1,19998330
32045,p19998330_s52741948_c8b41e34-9295f63e-b2edb096...,p19998330_s58626532_6c6f0868-10028db0-0270ff74...,0,19998330
32046,p19998330_s52741948_c8b41e34-9295f63e-b2edb096...,p19998330_s50007325_b5f405fb-36876897-aee19743...,0,19998330


In [19]:
train_data.to_csv(save_path + 'bnp_train.csv')
val_data.to_csv(save_path + 'bnp_validation.csv')
test_data.to_csv(save_path + 'bnp_test.csv')

In [30]:
zeros = 0
ones = 0
twos = 0 
total = 0
# for row in test_data.iterrows():
#     label = row[-1]['delta_bnp']
#     total += 1
#     if label == 0:
#         zeros += 1
#     elif label == 1:
#         ones += 1
#     else:
#         twos += 1
# for row in train_data.iterrows():
#     label = row[-1]['delta_bnp']
#     total += 1
#     if label == 0:
#         zeros += 1
#     elif label == 1:
#         ones += 1
#     else:
#         twos += 1
for row in val_data.iterrows():
    label = row[-1]['delta_bnp']
    total += 1
    if label == 0:
        zeros += 1
    elif label == 1:
        ones += 1
    else:
        twos += 1
print(zeros, ones, twos)

726 562 0
